Firslty we need to scrap data from Wikipedia using Scrapy

In [1]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 626.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.3 MB/s eta 0:00:00


In [2]:
# Install necessary packages
!pip install scrapy
!pip install w3lib

# Import required libraries
import scrapy
from scrapy.crawler import CrawlerProcess
import re

# Function to clean text data, removing unwanted characters and numbers inside brackets
def clean_text_list(text_list):
    """
    Cleans a list of text by removing unwanted characters, numbers in brackets, and extra spaces.

    Args:
        text_list (list): A list of strings that need to be cleaned.

    Returns:
        str: A comma-separated string of cleaned text. If the list is empty, returns "N/A".
    """
    cleaned = []
    for text in text_list:
        text = text.strip()  # Remove leading/trailing spaces
        # Remove all occurrences of standalone numbers inside brackets: [2], [3], etc.
        text = re.sub(r"\[\s*\d+\s*\]", "", text)
        # Remove unwanted keywords (CSS classes, special characters)
        if text and not any(x in text for x in ["mw-parser-output", "{", "}","[","]","1","2","3","4","5","6","7","8","9"]):
            cleaned.append(text)
    return ", ".join(cleaned) if cleaned else "N/A"  # Return cleaned text or "N/A" if no text is left

# Define the spider class that will crawl the Wikipedia page
class WikiSpider(scrapy.Spider):
    name = "wiki_spider"
    start_urls = ["https://en.wikipedia.org/wiki/List_of_highest-grossing_films"]  # Start URL for the spider

    def parse(self, response):
        """
        Parse the main page and extract the data for each movie listed in the table.

        Args:
            response (scrapy.http.Response): The response object returned by Scrapy for the start URL.

        Yields:
            dict: A dictionary containing movie details including rank, title, box office, year, and a link to the movie page.
        """
        table = response.xpath("(//table[contains(@class,'wikitable')])[1]")  # Extract the first table with the class 'wikitable'
        rows = table.xpath(".//tr")[1:]  # Skip the header row by selecting all rows except the first one

        for row in rows:
            # Extract rank, title, box office, and year from the row
            rank = row.xpath("./td[1]/text()").get()
            title = row.xpath(".//th/i/a/text() | .//th/span/i/a/text()").get()
            box_office = row.xpath(".//td[3]/text()").get()
            box_office = re.sub(r"[^\d.]", "", box_office)  # Clean up the box office value to retain only digits and decimal points
            year = row.xpath(".//td[4]/text()").get()
            relative_link = row.xpath("./th/i/a/@href | ./th/span/i/a/@href").get()

            if title and relative_link:
                full_link = response.urljoin(relative_link)  # Create the full URL for the movie page

                # Follow the link to the movie page and pass movie details to the next parsing method
                yield response.follow(full_link, self.parse_movie, meta={
                    'rank': rank.strip() if rank else "N/A",
                    'title': title.strip(),
                    'url': full_link,
                    'box_office': box_office.strip() if box_office else "N/A",
                    'year': year.strip() if year else "N/A"
                })

    def parse_movie(self, response):
        """
        Parse individual movie pages to extract director and country information.

        Args:
            response (scrapy.http.Response): The response object returned by Scrapy for the movie page.

        Yields:
            dict: A dictionary containing detailed movie information including directors and country of origin.
        """
        # Get movie details from the meta information passed in the previous method
        rank = response.meta['rank']
        title = response.meta['title']
        url = response.meta['url']
        box_office = response.meta['box_office']
        year = response.meta['year']

        # Extract director information from the movie page
        directors = response.xpath("//table[contains(@class, 'infobox')]//th[contains(text(), 'Directed by')]/following-sibling::td//text()[normalize-space() and not(parent::sup)]").getall()
        directors = clean_text_list(directors)  # Clean the list of director names

        # Extract country of origin from the movie page
        country = response.xpath("//table[contains(@class, 'infobox')]//th[contains(text(), 'Country') or contains(text(), 'Countries')]/following-sibling::td//text()[normalize-space() and not(parent::sup)]").getall()
        country = clean_text_list(country)  # Clean the list of countries

        # Yield the final data for each movie
        yield {
            "Rank": rank,
            "Title": title,
            "Release Year": year,
            "Directed by": directors,
            "Box Office Revenue": box_office,
            "Country of origin": country
        }

# Set up the Scrapy process and specify where to save the results
process = CrawlerProcess(settings={
    "FEEDS": {"output.json": {"format": "json"}},  # Save output to a JSON file
})

# Start the crawling process
process.crawl(WikiSpider)
process.start()  # Run the spider to collect data


INFO:scrapy.utils.log:Scrapy 2.12.0 started (bot: scrapybot)
2025-02-26 16:56:28 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-02-26 16:56:28 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2025-02-26 16:56:28 [scrapy.addons] INFO: Enabled addons:
[]
DEBUG:scrapy.utils.log:Using reactor: twisted.internet.epollreactor.EPollReactor
2025-02-26 16:56:28 [scrapy.utils.log] DEBUG: Using reactor: twi

Now we have our cleaned data in output.json. The next step is parse our data from json to database. I have chosen SQLite, since our data is small and we won't use database anyway😀

In [3]:
import sqlite3
import json
with open("output.json", "r", encoding="utf-8") as file:
    films_data = json.load(file)

In [4]:
conn = sqlite3.connect("films.db")
cursor = conn.cursor()

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office REAL,
    country TEXT
)
""")

We created our database and table films. So the next piece of code is for adding entities into our table

In [6]:
for film in films_data:
    cursor.execute("""
    INSERT INTO films (title, release_year, director, box_office, country)
    VALUES (?, ?, ?, ?, ?)
    """, (
        film.get("Title"),
        film.get("Release Year"),
        film.get("Directed by"),
        float(film.get("Box Office Revenue", 0)),  # Convert box office to float
        film.get("Country of origin")
    ))
conn.commit()
conn.close()

It is done, now we can use our database, but unfortunately we need to transform our database to json file again

In [7]:
# Connect to SQLite database
conn = sqlite3.connect("films.db")
cursor = conn.cursor()

# Fetch all data from the films table
cursor.execute("SELECT * FROM films")
films = cursor.fetchall()

# Get column names
columns = [description[0] for description in cursor.description]

# Convert data to JSON format
films_list = [dict(zip(columns, film)) for film in films]

# Save to a JSON file
json_file_path = "films_data.json"
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(films_list, json_file, indent=4)

print(f"Data exported to {json_file_path}")

# Close the connection
conn.close()

Data exported to films_data.json


Now we are finnaly done. The next part is to use this data in our web page on github.